In [51]:
from bs4 import BeautifulSoup
import re

In [73]:
with open('resumes/resume (3).html', 'r', encoding='utf-8') as file:
    html_content = file.read()

In [74]:
soup = BeautifulSoup(html_content, 'html.parser')

### Пол, возраст, город

In [75]:
gender = soup.find('span', {'data-qa': 'resume-personal-gender'}).text
age_text = soup.find('span', {'data-qa': 'resume-personal-age'}).text
city = soup.find('span', {'data-qa': 'resume-personal-address'}).text

# Используем регулярные выражения для извлечения числа из строки возраста
age = int(re.search(r'\d+', age_text).group())

# Выводим извлеченную информацию
print(gender, age, city)

Мужчина 28 Липецк


### Основная профессия резюме

In [76]:
profession_element = soup.find('span', class_='resume-block__title-text')
profession = profession_element.text.strip()
profession

'Агроном'

### Специализации

In [77]:
# Находим все элементы li, которые представляют специализации
specializations_elements = soup.find_all('li', class_='resume-block__specialization')

# Извлекаем текст из каждого элемента, содержащего специализацию
specializations = [element.text.strip() for element in specializations_elements]

# Выводим извлеченные специализации
specializations

['Агроном']

### Опыт работы в месяцах

In [78]:
experience_elements = soup.find_all('span', class_='resume-block__title-text_sub')[0].find_all('span')

# Инициализация переменных для годов и месяцев
years = 0
months = 0

# Проверяем каждый элемент на наличие информации о годах или месяцах
for element in experience_elements:
    text = element.text.strip()

    # Проверяем, содержит ли текст информацию о годах
    if 'лет' in text or 'год' in text:
        years = int(re.search(r'\d+', text).group())

    # Проверяем, содержит ли текст информацию о месяцах
    elif 'месяц' in text:
        months = int(re.search(r'\d+', text).group())

# Преобразуем годы в месяцы и суммируем с количеством месяцев
total_months = years * 12 + months
print(total_months)

63


In [79]:
# Находим основной блок с опытом работы
experience_main_block = soup.find('div', {'data-qa': 'resume-block-experience'})

# Проверяем, существует ли основной блок с опытом работы
if experience_main_block:
    # Находим все блоки с отдельными позициями опыта работы
    experience_blocks = experience_main_block.find_all('div', class_='resume-block-container')

    # Список для хранения данных опыта работы
    experiences = []

    for block in experience_blocks:
        # Извлекаем название должности
        position_element = block.find('div', {'data-qa': 'resume-block-experience-position'})
        position = position_element.text.strip() if position_element else 'Не указано'

        # Извлекаем описание работы
        description_element = block.find('div', {'data-qa': 'resume-block-experience-description'})
        description = description_element.text.strip() if description_element else 'Описание отсутствует'

        # Извлекаем период работы
        date_block = block.find_previous_sibling('div')
        print(date_block)
        if date_block:
            date_text = date_block.get_text(strip=True)
            date_parts = date_text.split('—')
            start_date = date_parts[0].strip()
            end_date = date_parts[1].strip() if len(date_parts) > 1 else 'Null'
        else:
            start_date = 'Не указано'
            end_date = 'Не указано'

        # Добавляем информацию в список
        experiences.append({
            'Название работы': position,
            'Описание': description,
            'Начало работы': start_date,
            'Конец работы': end_date
        })

    # Выводим информацию о каждом опыте работы
    for experience in experiences:
        print(experience)
else:
    print('Блок с опытом работы не найден')

None
{'Название работы': 'Агроном', 'Описание': '-', 'Начало работы': 'Не указано', 'Конец работы': 'Не указано'}
